In [ ]:
from happyalgos.utils.filtering import window_ranger
from happyalgos.activity.activity_counter import ActivityCounter
from sklearn.preprocessing import minmax_scale
import numpy as np
import pandas as pd
import scipy

General specs/loading data/windowing

In [ ]:
acc_df = pd.read_parquet("test_acc.parquet")
WINDOW, STRIDE = 60.0, 20.0
window_ranges = window_ranger(
    acc_df.iloc[0].etime,
    acc_df.iloc[-1].etime,
    STRIDE,
    WINDOW,
    )

act_obj = ActivityCounter( window = WINDOW, stride = STRIDE )

In [ ]:
for i, window in enumerate(window_ranges):
    acc_slice = acc_df[acc_df.etime.between( window[0], window[1] )]
    mag = np.sqrt( acc_slice.x.values ** 2 + acc_slice.y.values ** 2 + acc_slice.z.values ** 2 )
    
    f, pxx = scipy.signal.welch(
            mag,
            fs = 26,
            window = "hamming",
            nperseg = 256,
            noverlap = None,
            nfft = None,
            scaling = "density",
            detrend = "constant",
            average = "mean",
        )
    
    pxx_scaled = minmax_scale( pxx )
    
    sos = scipy.signal.butter(4, 0.7, fs = 26.0, output = "sos", btype = "highpass")
    x_hpf = scipy.signal.sosfilt(sos, acc_slice.x.values)
    y_hpf = scipy.signal.sosfilt(sos, acc_slice.y.values)
    z_hpf = scipy.signal.sosfilt(sos, acc_slice.z.values)
    mag_filt = np.sqrt(x_hpf ** 2 + y_hpf ** 2 + z_hpf ** 2)
    
    ratio = mag / mag_filt
    counts = act_obj.get_count( acc_slice.x.values, acc_slice.y.values, acc_slice.z.values, acc_slice.etime.values )

    # ratio features
    ratio_features = {
                    "ratio_mean": np.mean(ratio),
                    "ratio_std": np.std(ratio),
                    "ratio_median": np.median(ratio),
                    "ratio_skew": scipy.stats.skew(ratio),
                    "ratio_kurtosis": scipy.stats.kurtosis(ratio),
                    "ratio_cvar": np.var(ratio) / np.mean(ratio)
                    }

    # time domain features
    time_features = {
                    "counts"            : counts,
                    "mean_time"         : np.nanmean(mag),
                    "std_time"          : np.nanstd(mag),
                    "cvar_time"         : np.nanvar(mag) / np.nanmean(mag),
                    "median_time"       : np.nanmedian(mag),
                    "skewness_time"     : scipy.stats.skew(mag),
                    "kurtosis_time"     : scipy.stats.kurtosis(mag),
                    }


    freq_res = f[1] - f[0] 

    auc_0_3 = np.trapz( pxx[np.logical_and( f >= 0.0, f <= 3.0 )], dx = freq_res)

    high_range_auc = np.trapz( pxx[np.logical_and( f >= 7.0, f <= 13.0 )], dx = freq_res )
    low_range_auc = np.trapz( pxx[np.logical_and( f >= 0.0, f <= 5.0 )], dx = freq_res )
    hi_lo_unscaled = high_range_auc / low_range_auc

    high_range_auc = np.trapz( pxx_scaled[np.logical_and( f >= 7.0, f <= 13.0 )], dx = freq_res )
    low_range_auc = np.trapz( pxx_scaled[np.logical_and( f >= 0.0, f <= 5.0 )], dx = freq_res )
    hi_lo_scaled = high_range_auc / low_range_auc
    
    # freq domain feature
    freq_features = {
                    "max_pxx"           : np.nanmax(pxx),
                    "log_max_pxx"       : np.log( np.nanmax(pxx) ),
                    "skewness_freq"     : scipy.stats.skew( pxx ),
                    "kurotis_freq"      : scipy.stats.kurtosis( pxx ),
                    "mean_freq"         : np.mean( pxx ),
                    "std_freq"          : np.std( pxx ),
                    "cvar_freq"         : np.var(pxx) / np.mean(pxx),
                    "median_freq"       : np.median(pxx),
                    "hi_lo_scaled"      : hi_lo_scaled,
                    "hi_lo_unscaled"    : hi_lo_unscaled,
                    "auc_0_3"           : auc_0_3
                    }
    
    
